# Music Genre Classifier
--- 
## Imports

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andradaolteanu/gtzan-dataset-music-genre-classification")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\janmi\.cache\kagglehub\datasets\andradaolteanu\gtzan-dataset-music-genre-classification\versions\1


In [ ]:
print(type(path))

df = pd.read

<class 'str'>


In [ ]:
# Usual Libraries
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Librosa - for audio and music processing
import librosa
import librosa.display
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

---

## Explore Audio Data

In [ ]:
y, sr = librosa.load()